In [1]:
import cv2
import sys
import os
import time
import random
import numpy as np
import tensorflow as tf

In [5]:
SAVER_DIR = "train_saver/digits/"
SIZE = 576
CHANNELS = 1
WIDTH = 24
HEIGHT = 24
NUM_CLASSES = 10
license_num = ""
LETTERS_DIGITS = ("0","1","2","3","4","5","6","7","8","9")
 # 定义卷积函数
def conv_layer(inputs, W, b, conv_strides, kernel_size, pool_strides, padding):
    L1_conv = tf.nn.conv2d(inputs, W, strides=conv_strides, padding=padding)
    L1_relu = tf.nn.relu(L1_conv + b)
    return tf.nn.max_pool(L1_relu, ksize=kernel_size, strides=pool_strides, padding='SAME')
 
# 定义全连接层函数
def full_connect(inputs, W, b):
    return tf.nn.relu(tf.matmul(inputs, W) + b)

with tf.name_scope('input'):
    with tf.name_scope('input_x'):
        x = tf.placeholder(tf.float32, shape=[None, SIZE])
    with tf.name_scope('input_y'):
        y = tf.placeholder(tf.float32, shape=[None, NUM_CLASSES])

x_image = tf.reshape(x, [-1, WIDTH, HEIGHT, CHANNELS])

saver = tf.train.import_meta_graph("%smodel.ckpt-100.meta"%(SAVER_DIR))
with tf.Session() as sess:
    model_file=tf.train.latest_checkpoint(SAVER_DIR)
    print(model_file)
    
    saver.restore(sess, model_file)
    print(sess.graph.get_all_collection_keys())
    # 第一个卷积层
    W_conv1 = sess.graph.get_tensor_by_name("conv_layer1/W_conv1/W_conv1:0")
    b_conv1 = sess.graph.get_tensor_by_name("conv_layer1/b_conv1/b_conv1:0")
    conv_strides = [1, 1, 1, 1]
    kernel_size = [1, 2, 2, 1]
    pool_strides = [1, 2, 2, 1]
    L1_pool = conv_layer(x_image, W_conv1, b_conv1, conv_strides, kernel_size, pool_strides, padding='SAME')

    # 第二个卷积层
    W_conv2 = sess.graph.get_tensor_by_name("conv_layer2/W_conv2/W_conv2:0")
    b_conv2 = sess.graph.get_tensor_by_name("conv_layer2/b_conv2/b_conv2:0")
    conv_strides = [1, 1, 1, 1]
    kernel_size = [1, 1, 1, 1]
    pool_strides = [1, 1, 1, 1]
    L2_pool = conv_layer(L1_pool, W_conv2, b_conv2, conv_strides, kernel_size, pool_strides, padding='SAME')


    # 全连接层
    W_fc1 = sess.graph.get_tensor_by_name("fc_layer1/W_fc1/W_fc1:0")
    b_fc1 = sess.graph.get_tensor_by_name("fc_layer1/b_fc1/b_fc1:0")
    h_pool2_flat = tf.reshape(L2_pool, [-1, 12*12*64])
    h_fc1 = full_connect(h_pool2_flat, W_fc1, b_fc1)


    # dropout
    keep_prob = tf.placeholder(tf.float32)

    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    
    # readout层
    W_fc2 = sess.graph.get_tensor_by_name("fc_layer2/W_fc2/W_fc2:0")
    b_fc2 = sess.graph.get_tensor_by_name("fc_layer2/b_fc2/b_fc2:0")

    # 定义优化器和训练op
         
    
    dir = 'test_images/'          # 这里可以改成你自己的图片目录，i为分类标签
    for rt, dirs, files in os.walk(dir):
        for filename in files:
            # 读取图片
            if filename.endswith('.bmp'):
                filename = dir + filename
                img = cv2.imread(filename)
                imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
                #cv2.imshow('resize %s' % filename,img)
                imgResize = cv2.resize(imgGray,(WIDTH,HEIGHT))
                cv2.imshow('resize %s' % filename,imgResize)
                #cv2.imwrite(filename.replace('.png','.bmp'),imgResize)
                cv2.waitKey()
                #转换数据格式为（256，1）
                img_data = [[0]*SIZE for i in range(1)]

                img_data[0] =  imgResize.reshape(1,SIZE)
                print(img_data[0])
                result = sess.run(conv, feed_dict = {x: np.array(img_data[0]),keep_prob: 1.0})

                max1 = 0
                max2 = 0
                max3 = 0
                max1_index = 0
                max2_index = 0
                max3_index = 0
                for j in range(NUM_CLASSES):
                    if result[0][j] > max1:
                        max1 = result[0][j]
                        max1_index = j
                        continue
                    if (result[0][j]>max2) and (result[0][j]<=max1):
                        max2 = result[0][j]
                        max2_index = j
                        continue
                    if (result[0][j]>max3) and (result[0][j]<=max2):
                        max3 = result[0][j]
                        max3_index = j
                        continue

                license_num = license_num + LETTERS_DIGITS[max1_index]
                print ("概率：  [%s %0.2f%%]    [%s %0.2f%%]    [%s %0.2f%%]" % (LETTERS_DIGITS[max1_index],max1*100, LETTERS_DIGITS[max2_index],max2*100, LETTERS_DIGITS[max3_index],max3*100))
                print('/n')
                cv2.destroyAllWindows()
        print ("车牌编号是: 【%s】" % license_num)            


train_saver/digits/model.ckpt-100
INFO:tensorflow:Restoring parameters from train_saver/digits/model.ckpt-100
['summaries', 'train_op', 'trainable_variables', 'variables']
[[235 235 235 235 235 235 234 233 232 230 226 219 216 218 223 230 235 241
  241 241 242 242 236 233 233 233 233 233 234 235 236 234 231 218 203 182
  177 188 204 223 234 242 239 237 236 236 235 234 234 234 234 235 236 235
  233 225 216 202 183 153 147 166 190 216 228 236 233 231 231 232 237 239
  240 240 239 238 236 227 217 202 191 195 189 167 163 176 193 211 219 224
  228 231 235 239 240 241 242 235 223 210 197 190 184 189 195 204 202 179
  172 182 195 209 216 222 229 236 236 234 228 224 233 223 207 189 170 171
  175 198 216 219 211 184 174 183 197 213 221 227 232 235 231 225 216 210
  223 216 205 195 185 189 195 213 227 225 214 185 176 185 199 217 224 229
  231 233 229 226 220 216 220 219 217 217 218 220 222 227 228 226 214 185
  176 186 200 217 224 228 230 232 233 233 232 231 225 227 231 235 240 240
  240 235 230 

[[143 135 129 133 139 144 149 153 158 162 162 161 160 159 159 157 154 151
  149 146 140 137 135 135 143 136 133 140 145 148 149 150 154 157 156 156
  156 156 156 156 156 157 158 156 148 142 138 135 142 134 131 136 139 138
  135 134 134 132 131 130 133 137 140 144 147 153 159 163 156 147 139 135
  141 131 124 124 124 122 119 115 112 108 104 102 107 114 120 127 134 143
  152 159 158 148 140 134 141 129 119 113 110 107 105 102  99  95  92  92
   94  98 103 110 123 134 142 148 151 146 139 134 142 131 119 111 107 104
  102 102  99  98  97  97  97  97 100 105 117 128 135 138 139 137 133 130
  143 132 122 116 114 113 113 112 111 111 111 110 110 109 110 113 120 127
  131 130 129 129 129 127 142 132 122 119 118 119 119 119 119 119 119 120
  121 120 119 122 125 129 130 128 125 126 126 125 143 133 123 120 120 121
  121 123 123 124 126 127 129 128 128 129 130 130 127 125 122 123 123 124
  143 133 125 124 123 124 127 132 134 136 138 141 142 143 144 141 135 130
  126 121 118 120 121 123 142 133 125 

[[207 206 204 202 201 202 203 206 209 211 212 213 211 209 208 204 202 200
  199 199 200 201 202 203 205 203 201 201 202 202 204 205 206 208 208 208
  207 206 204 201 199 197 197 198 199 200 202 203 200 201 201 201 201 201
  201 200 200 201 201 201 202 203 204 205 206 206 204 201 198 200 201 201
  197 201 206 208 206 203 204 205 206 207 207 207 207 206 205 203 201 201
  201 201 201 200 201 200 198 205 212 217 220 220 218 218 217 221 224 228
  230 228 222 212 203 195 194 197 200 203 201 199 200 204 207 207 202 194
  186 180 179 186 198 211 224 230 231 227 212 198 186 185 190 203 204 200
  193 196 199 198 192 186 178 180 184 187 191 194 196 195 200 227 224 209
  187 176 175 195 205 210 201 202 202 199 196 196 202 205 205 199 200 202
  204 193 188 218 218 204 179 166 163 191 198 198 201 200 198 199 199 200
  199 199 200 196 197 199 203 201 202 218 209 190 165 153 155 189 194 191
  199 197 195 199 203 205 198 202 206 209 210 210 211 216 219 213 190 165
  151 154 167 191 199 200 208 206 204 

[[210 212 213 212 209 208 208 214 219 222 223 220 218 215 212 209 207 205
  204 205 205 205 206 207 210 213 213 212 209 208 208 214 221 224 225 222
  221 218 214 210 205 202 203 203 203 207 216 226 220 216 211 209 210 215
  219 212 206 206 210 218 226 228 226 215 195 180 198 203 205 207 212 219
  224 217 211 211 216 221 214 192 179 178 190 207 219 226 228 215 187 162
  189 201 205 206 207 212 217 214 214 216 220 217 186 160 158 169 185 198
  208 214 221 214 189 163 179 197 205 208 211 217 213 211 209 211 214 208
  164 149 166 190 204 204 210 210 219 217 195 164 169 194 208 212 215 220
  212 209 204 204 209 206 161 151 175 203 215 210 210 207 217 218 197 162
  157 188 205 210 212 217 211 208 203 205 213 215 179 165 177 197 209 209
  208 206 218 221 200 164 153 187 204 207 210 217 212 209 205 208 219 229
  212 193 189 195 201 205 204 207 220 222 201 166 156 191 204 205 207 216
  213 209 205 206 215 228 234 227 220 215 211 208 205 210 219 215 190 157
  154 190 203 201 200 205 210 208 204 

# 